# 25-26: 4369 -- PROGRAMMING FOR DATA ANALYTICS
## Big Project

One of the main challenges of this project (I found) was deciding on a suitable data subject to research and then attempting to **source** data.  
While I looked into data for Retail Sales, Traffic Accidents, Weather, etc (and browsing Kaggle for potential subjects) ... after several failed attempts, I found that *I needed a pursue a subject I was genuinely curious about.*  

I decided to look into the subject of **Trends in the use of Programming Languages** and to leverage the huge data repository of **GitHub** as a data source.  
  
I wanted to look at Programming Language Trends
* Analyze which languages are growing/declining over time
* Compare language popularity across different types of projects
* Visualize the shift from older to newer languages


GitHub Data Sources are available in :  
* GitHub API - Free tier allows decent access  
* GH Archive - Historical GitHub event data (billions of events!)  
* GitHub's public datasets on Google BigQuery  
* Pre-made datasets on Kaggle  

I looked into the GitHub API as a mechanism to extract the datasets I would require.

***
## The GitHub REST API with Python
### (... and its limitations 😖)

I experimented a while with Python scripts that collect GitHub Metadata and save to CSV files.  
https://blog.apify.com/python-github-api/  
https://melaniesoek0120.medium.com/how-to-use-github-api-to-extract-data-with-python-bdc61106a501  

I also investigated the use of the PyGithub Library.   
https://github.com/PyGithub/PyGithub  
https://pygithub.readthedocs.io/en/latest/introduction.html  
https://stackoverflow.com/questions/10625190/most-suitable-python-library-for-github-api-v3  
https://www.youtube.com/watch?v=QaURSdmP0o8  

But, this only highlighted how restrictive this API's "Rate Limit" can be.  
Look at script below (generated with help of Claude AI) that uses pyGithub module to attempt to look at the count of language specific  repositories created with a specific period  

[My Python Script for Extracting Github Data](./testing/github_api_test.py)  
Note: To use the PyGithub Module you will need to install PyGithub library with  
`pip install PyGithub`  

If you look at the results of this script (output to [this text file](./testing/Github_REST_API_Results.txt)), all language for all months return exactly same result ... "1,000 repos"  
**I am obviously hitting a limit of some sort here.**
   
A simplified version (with a hard-coded sample) is shown below where a reduced time interval of only 3 hours is used...   
It still returns a count close to the limit of 1000 repositories   

*Experimentation with values for language, start_date, and end_date highlight how restrictive this 1000 limit is.*

In [4]:
from github import Github, RateLimitExceededException, Auth
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Load environment variables from .env file
load_dotenv()

# Get the token from environment variables
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")


# Create authentication object
auth = Auth.Token(GITHUB_TOKEN)
# Initialize GitHub client with auth
g = Github(auth=auth)

# Verify authentication
try:
    user = g.get_user()
    print(f"Authenticated as: {user.login}")
except Exception as e:
    print(f"Authentication failed: {e}")

# Example usage 
language = 'Python'
start_date = '2024-03-01T00:00:00+00:00'
end_date = '2024-03-01T03:00:00+00:00'

# Build the API search query
query = f"language:{language} created:{start_date}..{end_date}"
# Search repositories with PyGithub
result = g.search_repositories(query=query)
# Get total count
total_count = result.totalCount
print(f"Count of {language} repos created between {start_date} and {end_date}: {total_count}")



Authenticated as: ngn73
Count of Python repos created between 2024-03-01T00:00:00+00:00 and 2024-03-01T03:00:00+00:00: 940


<font color="crimson">... **I needed to find an alternative (less restrictive) method to extracting Github Repository metadata.**</font>

***
## Google BigQuery + GitHub Archive  
  
https://console.cloud.google.com/  

Google is in collaboration with GitHub to release an incredible new open dataset on **Google BigQuery**  
The Google BigQuery Public Datasets program now offers a full snapshot of the content of more than 2.8 million open source GitHub repositories in BigQuery  
This provides a alternative mechanism to analyze the source code of almost 2 billion files with a simple (or complex) SQL query.   
  

https://cloud.google.com/blog/topics/public-datasets/github-on-bigquery-analyze-all-the-open-source-code  
https://hoffa.medium.com/github-on-bigquery-analyze-all-the-code-b3576fd2b150  
https://hoffa.medium.com/400-000-github-repositories-1-billion-files-14-terabytes-of-code-spaces-or-tabs-7cfe0b5dd7fd  
https://github.com/fhoffa/analyzing_github?tab=readme-ov-file  

The use of SQL is ideal as I am quite comfortable with the SQL syntax  
https://docs.cloud.google.com/bigquery/docs/reference/rest?apix=true   

<img src="../images/Goole_BigQuery_SQL.png" alt="Google BigQuery" width="600">

**These SQL scripts can be used to format/configure Datasets as required by the needs of my Data Analysis   
Results can then be saved as CSV or JSon files for further processing with Python.**



